In [1]:
import pandas as pd
import json
from datetime import datetime

### Xóa cột coefficient, language_id. Đổi tên cột assignment_id, problem_id, username

In [4]:

df = pd.read_csv(r'D:\DoAnMayHoc\score_regression\data\annonimized.csv')
df.rename(columns={
    "concat('it001',`assignment_id`)": 'assignment_id',
    "concat('it001',`problem_id`)": 'problem_id',
    "concat('it001', username)": 'username',
    "concat('it001',`language_id`)": 'language_id',
}, inplace=True)

df.drop(columns=['coefficient', 'language_id'], inplace=True)

df.to_csv(r'D:\DoAnMayHoc\score_regression\data\data_cleaned.csv', index=False)

### Lọc features

In [9]:
df = pd.read_csv(r'D:\DoAnMayHoc\score_regression\data\data_cleaned.csv')
df['created_at'] = df['created_at'].str.split().str[1]
df['updated_at'] = df['updated_at'].str.split().str[1]

df['created_at'] = pd.to_datetime(df['created_at'], format='%H:%M:%S')
df['updated_at'] = pd.to_datetime(df['updated_at'], format='%H:%M:%S')

# Sửa lỗi âm bằng cách giả sử updated_at hôm sau
df.loc[df['updated_at'] < df['created_at'], 'updated_at'] += pd.Timedelta(days=1)

# Tính thời gian
df['duration_seconds'] = (df['updated_at'] - df['created_at']).dt.total_seconds()

# Tóm tắt theo user
df_summary = df.groupby('username').agg(
    total_submissions=('username', 'count'),
    total_problems=('problem_id', 'nunique'),
    total_assignments=('assignment_id', 'nunique'),
    total_final=('is_final', 'sum'),
    total_score_status=('status', lambda x: (x == 'SCORE').sum()),
    avg_duration_seconds=('duration_seconds', 'mean')
).reset_index()


df_summary.to_csv(r'D:\DoAnMayHoc\score_regression\data\user_feature.csv', index=False)



In [8]:
df_features = pd.read_csv(r'D:\DoAnMayHoc\score_regression\data\user_feature.csv') 
df_scores = pd.read_csv(r'D:\DoAnMayHoc\score_regression\data\qt-public.csv')
df_scores.rename(columns={'hash': 'username'}, inplace=True)

# 1. user_without_features: users có trong features nhưng không có trong scores
users_features = set(df_features['username'])
users_scores = set(df_scores['username'])

users_without_scores = users_features - users_scores  # tập user có trong features nhưng không trong scores

df_user_without_features = df_features[df_features['username'].isin(users_without_scores)]

# 2. user_with_features: users có trong cả 2 file, lưu toàn bộ cột (merge trên username)
df_user_with_features = pd.merge(df_scores, df_features, on='username', how='inner')

# Lưu ra file
df_user_without_features[['username']].to_csv(r'D:\DoAnMayHoc\score_regression\data\qt\user_without_features.csv', index=False)
df_user_with_features.to_csv(r'D:\DoAnMayHoc\score_regression\data\qt\user_with_features.csv', index=False)

In [2]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Đọc file
df = pd.read_csv(r'D:\DoAnMayHoc\score_regression\data\qt\user_with_features.csv')

# Giả sử cột điểm cần dự đoán là 'diemqt'
target_col = 'diemqt'  # đổi theo cột điểm thực tế của bạn

# Lọc features: bỏ cột username và cột target
X = df.drop(columns=['username', target_col])
y = df[target_col]

# Tách train/test 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = pd.to_numeric(y_train, errors='coerce')

# Lọc bỏ những dòng mà y_train là NaN, đồng bộ X_train theo
mask = y_train.notna()
X_train = X_train[mask]
y_train = y_train[mask]

# Khởi tạo và train model
model = LinearRegression()
model.fit(X_train, y_train)
model.fit(X_train, y_train)

# Dự đoán trên tập test
y_pred = model.predict(X_test)

# Đánh giá
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse:.4f}')
print(f'R^2 Score: {r2:.4f}')





Mean Squared Error: 2.4704
R^2 Score: -0.0434


In [6]:
# 1. Đọc dữ liệu
df_features = pd.read_csv(r'D:\DoAnMayHoc\score_regression\data\user_feature.csv') 
df_without_features = pd.read_csv(r'D:\DoAnMayHoc\score_regression\data\qt\user_without_features.csv')

df_to_predict = df_features[df_features['username'].isin(df_without_features['username'])].copy()

# Xác định các cột đặc trưng
feature_cols = ['total_submissions', 'total_problems', 'total_assignments',
                'total_final', 'total_score_status', 'avg_duration_seconds']

# Dự đoán
df_to_predict['predicted_score'] = model.predict(df_to_predict[feature_cols])

# Làm tròn đến 0.5 và giới hạn tối đa là 10
df_to_predict['predicted_score'] = (df_to_predict['predicted_score'] * 2).round() / 2
df_to_predict['predicted_score'] = df_to_predict['predicted_score'].clip(upper=10)
df_to_predict[['username', 'predicted_score']].to_csv(r'D:\DoAnMayHoc\score_regression\data\qt\qt_submit_2.csv', index=False)

